## Table of Contents

* [0 Grid information](#0_Grid_information)

In [ ]:
######## LOBAFLEX ###########
# from lobaflex.analysis import analysis
# from lobaflex.tools.draw import *
from lobaflex.opt.feeder_extraction import get_flexible_loads
from lobaflex.tools.tools import get_config, get_files_in_subdirs
from lobaflex.opt.dispatch_integration import integrate_dispatch
from edisgo.edisgo import import_edisgo_from_files

######## DIRS ##########
from lobaflex import config_dir, data_dir, results_dir
######## DATA ###########
import re
import pandas as pd
from pathlib import Path
######## PLOTTING ###########
import matplotlib
# matplotlib.use('TkAgg')
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# # set seaborn style
# sns.set()
# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

# comment to make plots interactive
pio.renderers.default = "svg"


In [ ]:
cfg_o = get_config(path=config_dir / ".opt.yaml")
# cfg_g = get_config(path=config_dir / ".grids.yaml")

## 0 Grid information<a class="anchor" id="0_Grid_information"></a>

# Definitions

In [ ]:
# grid_path = results_dir / run_id / str(grid_id) / "minimize_loading" / "mvgd"
grid_path = Path(import_dir) / "mvgd"
# grid_path = import_dir
# grid_path = results_dir / run_id / str(grid_id) / "min_reinforce"
base_objective =  grid_path.parent.name
# results_path = results_dir / run_id / str(grid_id) / "potential" / "60_pct_reinforced"
results_path = results_dir / run_id / str(grid_id) / "potential" / base_objective

list_of_results = get_files_in_subdirs(path=results_path, pattern="*.csv")

In [ ]:
selected_list = [i for i in list_of_results if "concat" in i]

In [ ]:
def get_all_attribute_values_for_keyword(keyword):
    
    objectives = ["maximize_grid_power",
                  "minimize_grid_power",
                  "maximize_energy_level",
                  "minimize_energy_level",
                 ]
    
    colors_dict = {
        "load": "#db3b3c",
        "reactive_load": "#951b1c",
        "feedin": "#4bce4b",
        "reactive_feedin": "#1c641c",
        "hp": "#8c564b",
        "ev": "#1f77b4",
        "tes": "#d6aa27",
        }
    
    data = dict().fromkeys(objectives)
    
    keyword_files = [i for i in selected_list if keyword in i and not 'slack_initial' in i]

    
    for obj in objectives:
        obj_files = [i for i in keyword_files if obj in i]
        files = [i for i in obj_files if keyword in i]

        obj_dict = {}
        for i, file in enumerate(files):
            attr = re.findall(fr'{keyword}_(.*).csv', file)[0]
            df = pd.read_csv(file, index_col=0, parse_dates=True)
            df = df.sum(axis=1).rename(f"{keyword} {attr} [MW]")
            obj_dict[attr] = df
        data[obj] = obj_dict
            
    return data

In [ ]:
def plot_all_attributes_for_keyword(keyword, edisgo_obj=None):
    
    if edisgo_obj is not None:
        p_set = edisgo_obj.topology.loads_df.loc[
            edisgo_obj.topology.loads_df["opt"]].groupby("type")["p_set"].sum()
        upper_power = edisgo_obj.electromobility.flexibility_bands["upper_power"].sum(axis=1)
    
    objectives = ["maximize_grid_power",
                  "minimize_grid_power",
                  "maximize_energy_level",
                  "minimize_energy_level",
                 ]
    
    colors_dict = {
        "load": "#db3b3c",
        "reactive_load": "#951b1c",
        "feedin": "#4bce4b",
        "reactive_feedin": "#1c641c",
        "hp": "#8c564b",
        "ev": "#1f77b4",
        "tes": "#d6aa27",
        "hp_el": "#8c564b",
        }
    
    keyword_files = [i for i in selected_list if keyword in i and not 'slack_initial' in i]

    if len(keyword_files) == 0:
        print(f"No {keyword} values.")

    else:

        fig = make_subplots(rows=len(objectives), subplot_titles=objectives, vertical_spacing=0.05)

        for subplot, obj in enumerate(objectives):
            obj_files = [i for i in keyword_files if obj in i]
            files = [i for i in obj_files if keyword in i]

            if len(files) == 0:
                fig.add_annotation(x=2, y=5, text=f"No values", showarrow=False, font=dict(size=16),
                                  row=subplot+1, col=1,)
                continue
            traces = []
            for i, file in enumerate(files):
                attr = re.findall(fr'{keyword}_(.*).csv', file)[0]
                df = pd.read_csv(file, index_col=0, parse_dates=True)
                df = df.sum(axis=1).rename(f"{keyword} {attr} [MW]")

                
                trace = go.Scatter(x=df.index, y=df, mode="lines", name=attr, line_color=colors_dict[attr],

    #                                showlegend=True if not subplot else False,
    #                                legendgroup=attr,

                                   legendgroup=obj,
                                   legendgrouptitle_text=obj
                                  )
                fig.add_trace(trace, row=subplot+1, col=1,)
            
 
            if keyword == "charging" and edisgo_obj is not None:
                # draw p_set line
                trace = go.Scatter(x=df.index, y=[p_set["heat_pump"]], mode="lines", name="hp p_set",
                                   line_color=colors_dict["hp_el"],
                                   line=dict(dash='dot'),

#                                showlegend=True if not subplot else False,
#                                legendgroup=attr,

                               legendgroup=obj,
                               legendgrouptitle_text=obj
                              )
                fig.add_trace(trace, row=subplot+1, col=1,)
                
                # draw ev upper power line
                trace = go.Scatter(x=df.index, y=upper_power, mode="lines", name="ev upper_power",
                                   line_color=colors_dict["ev"],
                                   line=dict(dash='dot'),

#                                showlegend=True if not subplot else False,
#                                legendgroup=attr,

                               legendgroup=obj,
                               legendgrouptitle_text=obj
                              )
                fig.add_trace(trace, row=subplot+1, col=1,)

                
                

        fig.update_layout(
            title=f"{keyword}: {run_id} - {grid_id}",
            width=1000,
            height=2000,
            margin=dict(t=30, b=30, l=30, r=30),
            showlegend=True,
        )
        if "energy" in keyword:
            ylabel = "Energy in MWh"
        else:
            ylabel = "Power in MWh"
        fig.update_yaxes(title_text=ylabel)

        fig.show()



In [ ]:
def heatmap_energy_level(keyword):

    
    keyword_files = [i for i in selected_list if keyword in i and not 'slack_initial' in i]
    
    if len(keyword_files) == 0:
        print(f"No {keyword} values.")

    else:
        
        pattern = fr"potential/\w+/(.*)/concat/\d+_energy_level_(.*).csv"
        keys = [re.search(pattern, string=i).groups() for i in keyword_files]
        
        files = pd.DataFrame(
            keys, index=keyword_files, columns=["objective", "attribute"]
        ).sort_values(["attribute", "objective"])

        subplot_titles = files["objective"].values + "_" + files["attribute"].values
        
        fig = make_subplots(rows=len(files),
                            subplot_titles=subplot_titles,
                            vertical_spacing=0.05,
                           )

        traces = []
        for i, file in enumerate(files.index):
            attr = re.findall(fr'{keyword}_(.*).csv', file)[0]
            df = pd.read_csv(file, index_col=0, parse_dates=True)
            df = df.sum(axis=1).rename(f"{keyword} {attr} [MW]")
            data = [df.iloc[i:].loc[::24] for i in range(24)]
            trace = go.Heatmap(z=data, colorscale='Viridis', showscale=False)
            fig.add_trace(trace, row=i+1, col=1)


        fig.update_layout(
            title=f"{keyword}: {run_id} - {grid_id}",
            width=1000,
            height=2000,
            margin=dict(t=30, b=30, l=30, r=30),
            showlegend=True,
             coloraxis_showscale=True
        )
        if "energy" in keyword:
            ylabel = "Energy in MWh"
        else:
            ylabel = "Power in MWh"
        fig.update_yaxes(title_text=ylabel, tickmode='linear', dtick=4)
        fig.update_xaxes(title_text=ylabel, tickmode='linear', dtick=1)

        fig.show()



# Import grid

In [ ]:
edisgo_obj = import_edisgo_from_files(
    grid_path,
    import_topology=True,
    import_timeseries=True,
    import_heat_pump=True,
    import_electromobility=True,
)

# Plots
## curtailment

In [ ]:
plot_all_attributes_for_keyword(keyword="curtailment")

## Slacks

In [ ]:
plot_all_attributes_for_keyword(keyword="slack")

## Energy

In [ ]:
plot_all_attributes_for_keyword(keyword=f"{grid_id}_energy_level")

In [ ]:
heatmap_energy_level(keyword=f"{grid_id}_energy_level")

heatmap

- tes upper energy level constraint needs to be removed

## Charging

- diff zu pset hp
- diff zu upper power ev

In [ ]:
plot_all_attributes_for_keyword(keyword="charging", edisgo_obj=edisgo_obj)

- high initial charging bei step 1
    - hp
    - tes
- Speicher werden direkt voll gemacht

- konstantes laden der TES wegen verlusten
- warum Einbruch am Ende?

## Charging + feedin

# Plot dumb + opt charging

# Plot energy level + charging